In [1]:
import pandas as pd
import ast

In [2]:
df = pd.read_excel("data/diyetkolik_recipes.xlsx", index_col=0)

In [3]:
def parse_string_as_list(string):
    try:
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        return string

In [4]:
df.Ingredients= df.Ingredients.apply(parse_string_as_list)

In [5]:
ing_counts = dict()

for row in df.Ingredients:
    for ing in row:
        if ing not in ing_counts.keys():
            ing_counts[ing] = 1
        else:
            ing_counts[ing] += 1     

In [6]:
for ing, count in list(ing_counts.items()):
    if count < 15:
        del ing_counts[ing]

In [7]:
new_df = pd.DataFrame(0, index=df.index, columns=ing_counts.keys())

In [8]:
for index, recipe in df.iterrows():
    ingredients = recipe["Ingredients"]
    for ingredient in ingredients:
        if ingredient in new_df.columns.values:
            new_df.at[index,ingredient] = 1

In [9]:
new_df = new_df[ing_counts.keys()][0:1325]

In [10]:
from finch import FINCH
c, num_clust, req_c = FINCH(new_df)

Partition 0: 243 clusters
Partition 1: 42 clusters
Partition 2: 7 clusters
Partition 3: 4 clusters
Partition 4: 3 clusters


/Users/emrekuru/Downloads/Contrastive-Learning/env/lib/python3.11/site-packages/finch/finch.py:13: UserWarning: pynndescent not installed: No module named 'pynndescent'
  warnings.warn('pynndescent not installed: {}'.format(e))


In [11]:
num_clust

[243, 42, 7, 4, 3]

In [12]:
new_df["cluster_labels"] = pd.DataFrame(c)[2].values